In [0]:
import numpy as np
import matplotlib.pyplot as plt
import glob as gb
from matplotlib import animation
from os.path import exists
from google.colab import files
from PIL import Image

In [0]:
def gen_img(x_sehat, x_terinfeksi, x_sembuh, y_sehat , y_terinfeksi, y_sembuh, n_terinfeksi, n_sembuh, sembuh, terinfeksi, hari):
  
  x = (x_sehat, x_terinfeksi, x_sembuh)
  y = (y_sehat, y_terinfeksi, y_sembuh)

  groups = ("Sehat","Terinfeksi", "Sembuh")
  colors = ("blue","red", "green")
  markers = ("o","o","o")
  sizes = (50,100,100)
  
  fig = plt.figure(figsize=(8, 8))
  ax = fig.add_subplot(1, 1, 1)
  plt.grid

  for x, y, color, group, markers, sizes in zip(x, y, colors, groups, markers, sizes):
    ax.scatter(x, y, alpha=0.8, c=color, edgecolors=color, s=sizes, label=group,
               marker=markers )

  if hari == 1:
    plt.title('Hari ke-'+str(hari)+'     Terinfeksi = '+str(n_terinfeksi-terinfeksi)+' orang  (+0 orang)     Sembuh = 0 orang  (+0 orang)')
  elif hari>1:
    plt.title('Hari ke-'+str(hari)+'     Terinfeksi = '+str(n_terinfeksi-terinfeksi)+' orang  (+'+str(terinfeksi)+' orang)'+
              '     Sembuh = '+str(n_sembuh)+' orang  (+'+str(sembuh)+' orang)')

  plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.05),
          fancybox=True, shadow=True, ncol=5)
  plt.show()

  if exists('image/hari.png') and (hari != 1):
      fig.savefig('image/hari{}.png'.format(int(hari)))
      files.download('image/hari{}.png'.format(int(hari)))
      
  else:
      fig.savefig('image/hari.png')
      files.download('image/hari.png')